### Exercícios práticos 04 - Syanne Tavares

#### Resolução de sistemas lineares

- Podem ter 3 situações

1. Solução única -> retas concorrentes -> se tocam em 1 único ponto
2. Infinitas soluções -> retas coicidentes - pode ser qualquer tipo de valor
3. Sem solução -> Reta paralelas 

- 2 formas de resolver 
1. Métodos Diretos 
2. Métodos iterativos


## Exercício 01 - 

Implemente todas as técnicas de solução de sistemas lineares vistas em aula e faça uma comparação entre elas para as seguintes situações

# Métodos Diretos
## Fornecem soluções exatas com um número finito de operações. Podem ocorrer erros de arredondamento


# 1. Eliminação de Gauss
Resumidamente -> transforma o sistema em um sistema equivalente do tipo triangular superior e resolve a partir disso

In [3]:
import numpy as np
import pandas as pd

# Resolve o sistema triangular superior.
def SistemaTriangularSuperior(Ax,b):
  """Função que resolve um sistema linear do tipo  𝐴𝑥 = b """
  n = len(b) #tamanho do vetor b
  x = [0]*n #cria um vetor de mesmo tamanho de b com valores zerados

  x[n-1] = b[n-1]/Ax[n-1][n-1]

  for i in list(range(n-1,0,-1)):
    s = 0
    for j in list(range(i+1,n+1,1)):
      s = s + Ax[i-1][j-1]*x[j-1]

    x[i-1] = (b[i-1]-s)/(Ax[i-1][i-1])

  return x

# # Dados do problema
# U = [[3,     2,     4], 
#      [0, (1/3), (2/3)],
#      [0,     0,    -8]]
# b = [1, (5/3), 0]
# x = SistemaTriangularSuperior(U,b)
# print("x = ",x)


def eliminacao(A,b):
  n = len(b)
  x = [0]*n

  # Calculo dos pivos.
  for k in list(range(1,n,1)):
    # Calculo dos multiplicadores.
    for i in list(range(k+1,n+1,1)):
      m = A[i-1][k-1]/A[k-1][k-1]
      A[i-1][k-1] = 0
      b[i-1] = b[i-1] - m*b[k-1]
      # Atualizar demais valores da linha
      for j in list(range(k+1,n+1,1)):
        A[i-1][j-1] = A[i-1][j-1]-m*A[k-1][j-1] 

  return A,b

# # Teste com dados.
# Ai = [[3, 2, 4],
#       [1, 1, 2],
#       [4, 3,-2]]

# bi = [1, 2, 3]

# # Transformando em sist. equiv.
# U,b = eliminacao(Ai,bi)
# print("U =",U)
# print("b =",b)

def EliminacaoDeGauss(Ax,b):
  Ax2,b2 = eliminacao(Ax,b)
 
  print("depois da eliminação\n Ax =  ",Ax2,"\nb = ",b2)
  x = SistemaTriangularSuperior(Ax2,b2)
  return(x)



In [4]:
Ax = [[3, 2, 4],
      [1, 1, 2],
      [4, 3,-2]]
b = [1, 2, 3]

print(EliminacaoDeGauss(Ax,b))

depois da eliminação
 Ax =   [[3, 2, 4], [0, 0.33333333333333337, 0.6666666666666667], [0, 0, -8.0]] 
b =  [1, 1.6666666666666667, -4.440892098500626e-16]
[-3.0, 5.0, 5.551115123125783e-17]


# Fatoração LU com pivoteamento parcial

#### O processo de fatoração consistem em decompor a matriz A em um produto de dois ou mais fatores
▪ De forma que seja mais fácil resolver o sistema em etapas

In [51]:
def formata_matriz(M):
  m = len(M) # número de linhas
  n = len(M[0]) # número de colunas
  s = ""
  for i in range(m):
      for j in range(n):
          s += "%9.3f " % M[i][j]
      s +=  "\n"
  return s

# Resolve o sistema triangular inferior.
def SistemaTriangularInferior(L,b):
  n = len(b)
  y = [0]*n
  
  for i in list(range(1,n+1,1)):
    s = 0
    for j in list(range(1,i,1)):
      s = s + L[i-1][j-1]*y[j-1]

    y[i-1] = b[i-1] - s

  return y

def LU(A,b):
  n = len(A)
  x = [0]*n

  # Calculo dos pivos.
  for k in list(range(1,n,1)):
    # Calculo dos multiplicadores.
    for i in list(range(k+1,n+1,1)):
      m = A[i-1][k-1]/A[k-1][k-1]
      A[i-1][k-1] = m
      # Atualizar demais valores da linha
      for j in list(range(k+1,n+1,1)):
        A[i-1][j-1] = A[i-1][j-1]-m*A[k-1][j-1] 

  return A

# Função de pivoteamento parcial     
def partialPivoting(m, i):

    if(i != len(m)-1):
        divisor = m[i][i]

        for row in range(i+1, len(m)):
            n = m[row][i]
            for col in range(i, len(m)):
                m[row][col] -= (n / divisor) * m[i][col]
            m[row][i] = n / divisor

def pivoteamentoParcial(A):
  n = len(A)

  for k in range(1,n-1):
    pv = abs(A[k][k])
    Lpivo = k

    for i in range(k+1,n):
        if abs(A[i][k])> pv :
          pv = abs(A[i][k])
          Lpivo = i
  
    if pv == 0: 
      print("Matriz Singular")
      break

    if Lpivo != k : # se for diferente então tem que realizar a permutação
      for j in range (1,n):
        Troca = A[k][j]
        A[k][j] = A[Lpivo][j]
        A=[Lpivo][j]=Troca
      
      for i in range (k+1,n):
         m = A[i][k]/A[k][k]
         A[i][k] = m
         for j in range(k+1,n):
            A[i][j]=A[j][j] - m*A[k][j]

  return A

def LU_PivParc(A,b):
  n = len(A)
  A = pivoteamentoParcial(A)
  # resolução dos sistemas triangulares

  # Calculo dos pivos.
  for k in list(range(1,n,1)):
    # Calculo dos multiplicadores.
    for i in list(range(k+1,n+1,1)):
      m = A[i-1][k-1]/A[k-1][k-1]
      A[i-1][k-1] = m
      # Atualizar demais valores da linha
      for j in list(range(k+1,n+1,1)):
        A[i-1][j-1] = A[i-1][j-1]-m*A[k-1][j-1]
  
  y = SistemaTriangularInferior(A,b)
  x = SistemaTriangularSuperior(A,y)

  return x
  

In [52]:
# Teste com dados.
Ai = [[3, -4, 1],
      [1, 2, 2],
      [4, 0, 3]]

bi = [9, 3, -2]

# Obtendo os fatores L e U.
A = LU(Ai,bi)
print("A =",formata_matriz(A))
y = SistemaTriangularInferior(A,bi)
print("y =",y)
x = SistemaTriangularSuperior(A,y)
print("x =",x)

print("Resolução final")

print(LU_PivParc(Ai,bi))

A =     3.000    -4.000     1.000 
    0.333     3.333     1.667 
    1.333     1.600    -1.000 

y = [9, 0.0, -14.0]
x = [-10.999999999999998, -6.999999999999999, 13.999999999999996]
Resolução final
[1.2821576763485483, -0.6016597510373441, 2.746887966804979]
